In [9]:
import pandas as pd
import numpy as np
import scipy
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import math
import pickle
from annoy import AnnoyIndex
from tqdm import tqdm
import torch.sparse

patient_icd_path="/data1/andrew/meng/mixehr/data/Mimic/andrew_outputs/PATIENT_ICD_BINARY.csv"

In [ ]:
patient_icd_df = pd.read_csv(patient_icd_path, sep=' ')

In [ ]:
patient_data = patient_icd_df.drop("SUBJECT_ID", axis=1).as_matrix()

In [ ]:
subject_ids = pd.read_csv("PATIENT_ICD_SUBJECT_IDS.csv")

In [ ]:
patient_data_sparse[0:]

In [ ]:
patient_icd_df = pd.read_csv(patient_icd_path, sep=' ')

patient_data_sparse_coo = coo_matrix(patient_data)
patient_data_sparse_csc = csc_matrix(patient_data)
patient_data_sparse_csr = csr_matrix(patient_data)
patient_data_norm = normalize(patient_data_sparse_csr)

# pickle.dump(patient_data_sparse_coo, open("PATIENT_ICD_BINARY_SPARSE_COO.p", 'wb'))
# pickle.dump(patient_data_sparse_csc, open("PATIENT_ICD_BINARY_SPARSE_CSC.p", 'wb'))
# pickle.dump(patient_data_sparse_csr, open("PATIENT_ICD_BINARY_SPARSE_CSR.p", 'wb'))
# pickle.dump(patient_data_norm, open("PATIENT_ICD_BINARY_SPARSE_CSR_NORM.p", 'wb'))

In [2]:
patient_data_sparse_coo = pickle.load(open("PATIENT_ICD_BINARY_SPARSE_COO.p", 'rb'))
patient_data_sparse_csc = pickle.load(open("PATIENT_ICD_BINARY_SPARSE_CSC.p", 'rb'))
patient_data_sparse_csr = pickle.load(open("PATIENT_ICD_BINARY_SPARSE_CSR.p", 'rb'))
patient_data_norm_csr = pickle.load(open("PATIENT_ICD_BINARY_SPARSE_CSR_NORM.p", 'rb'))

In [ ]:
sparse_sim_csr = patient_data_sparse_csr[:batch_size].dot(patient_data_sparse_csr.T)

In [ ]:
%%timeit -n 1 -r 1

batch_size = 40

batch = patient_data_norm_csr[:5000]
full = patient_data_norm_csr

batch.dot(full.T)

In [ ]:
sparse_sim_coo = coo_matrix(spare_sim_csr)

In [ ]:
sparse_sim_coo.row

In [ ]:
sparse_sim_coo.col
idxs = np.vstack((sparse_sim_coo.row, sparse_sim_coo.col))
idxs

In [ ]:
i = torch.LongTensor(idxs)
v = torch.FloatTensor(sparse_sim_coo.data)
shape = sparse_sim_coo.shape

torch.sparse.FloatTensor(i, v, torch.Size(shape)).to_dense()

In [21]:
def build_annoy_index(sparse_csr_matrix: scipy.sparse, 
                      filename: str,
                      distance_metric: str = 'angular',
                      n_trees: int = 10):
    if distance_metric in ['cosine', 'angular']:
        distance_metric = 'angular'
        
    feat_size = sparse_csr_matrix.shape[-1]
    annoy = AnnoyIndex(feat_size, distance_metric)
    for i in range(sparse_csr_matrix.shape[0]):
        annoy.add_item(i, sparse_csr_matrix[i].toarray()[0])    
    annoy.build(n_trees)
    annoy.save("{}.ann".format(filename))

def load_annoy_index(feat_size: int, 
                     filename: str, 
                     distance_metric: str="angular"):
    a = AnnoyIndex(feat_size, distance_metric)
    a.load("{}.ann".format(filename))
    return a

def compute_sparse_similarity(batch: scipy.sparse, 
                              full: scipy.sparse, 
                              metric: str="cosine"):
    assert batch.shape[-1] == full.shape[-1], "batch and data feature dimensions do not match"
    
    if metric == "cosine":
        batch = normalize(csr_matrix(batch))
        full = normalize(csr_matrix(full))
        return batch.dot(full.T)
    
def construct_knn_graph(batch: scipy.sparse, 
                         full: scipy.sparse,
                         filename: str = "PATIENT_ICD_ANNOY_INDEX_ANGULAR",
                         distance_metric: str="cosine"):
    
    feat_size = batch[0].shape[-1]
    try:
        annoy = load_annoy_index(feat_size, filename)
    except OSError:
        annoy = build_annoy_index(full, filename, distance_metric)
        
    annoy = load_annoy_index(full.shape[-1], filename)
    sims = np.zeros((batch.shape[0], full.shape[0]))
    for i, row in enumerate(batch):
        nn_idxs = annoy.get_nns_by_item(i, 100)
        distances = [annoy.get_distance(i, nn) for nn in nn_idxs]
        sims[i][nn_idxs] = distances
        
    return sims

In [24]:
construct_knn_graph(batch, full)

3680

In [ ]:
annoy = load_annoy_index(patient_data_norm_csr, None)

In [13]:
subject_ids = pd.read_csv("PATIENT_ICD_SUBJECT_IDS.csv")
subject_ids

,SUBJECT_ID
0,109
1,112
2,113
3,114
4,115
...,...
46515,97164
46516,97484
46517,97488
46518,97492


In [14]:
subject_ids.iloc[0]["SUBJECT_ID"]

109